# Burst Extractor: Sentinel-1 Burst IDs From Sentinel-1 Scene IDs

Sentinel-1 scenes experience frame shift. Stacks of scenes are not guaranteed to be coregistered, and usually are not.

Sentinel-1 bursts are geostationary, and stacks comprised of bursts are coregistered. 

OPERA data are delivered as bursts. 

This notebook uses the ASF Burst Extractor to identify burst IDs for a given Sentinel-1 scene. 

<div class="alert alert-info" style="display: flex; align-items: center; font-family: 'Times New Roman', Times, serif; background-color: #d1ecf1;">
  <div style="display: flex; align-items: center; width: 10%;">
    <a href="https://github.com/ASFOpenSARlab/opensarlab_OPERA-CSLC_Recipe_Book/issues">
      <img src="https://opensarlab-docs.asf.alaska.edu/opensarlab-notebook-assets/logos/github_issues.png" alt="GitHub logo over the word Issues" style="width: 100%;">
    </a>
  </div>
  <div style="width: 95%;">
    <b>Did you find a bug? Do you have a feature request?</b>
    <br/>
    Explore GitHub Issues on this Jupyter Book's GitHub repository. Find solutions, add to the discussion, or start a new bug report or feature request: <a href="https://github.com/ASFOpenSARlab/opensarlab_OPERA-RTC-S1_Recipe_Book/issues">opensarlab_OPERA-RTC-S1_Recipe_Book Issues</a>
  </div>
</div>

<div class="alert alert-info" style="display: flex; align-items: center; justify-content: space-between; font-family: 'Times New Roman', Times, serif; background-color: #d1ecf1;">
  <div style="display: flex; align-items: center; width: 10%; margin-right: 10px;">
    <a href="mailto:uso@asf.alaska.edu">
      <img src="https://opensarlab-docs.asf.alaska.edu/opensarlab-notebook-assets/logos/ASF_support_logo.png" alt="ASF logo" style="width: 100%">
    </a>
  </div>
  <div style="width: 95%;">
    <b>Have a question related to SAR, OPERA RTCs, or ASF data access?</b>
    <br/>
    Contact ASF User Support: <a href="mailto:uso@asf.alaska.edu">uso@asf.alaska.edu</a>
  </div>
</div>

In [ ]:
import getpass
import requests

import asf_search as disco

In [ ]:
username = input('Username:')
password = getpass.getpass('Password:')

try:
    session = disco.ASFSession().auth_with_creds(username, password)
except disco.ASFAuthenticationError as e:
    print(f'Auth failed: {e}')
else:
    print('Success!')

In [ ]:
scene = input("Enter a Sentinel-1 SLC ID")

In [ ]:
beam_modes = ['IW1', 'IW2', 'IW3']
max_bursts_per_swath = range(15)

orbital_path = f"T{disco.granule_search(scene)[1].properties['pathNumber']:03d}"

run = True
while run:
    bursts = []
    run = False
    for beam_mode in beam_modes:
        for i in max_bursts_per_swath:
            url = f'https://sentinel1-burst.asf.alaska.edu/{scene}/{beam_mode}/VV/{i}.xml'
            response = session.get(url)
            status = response.status_code
            if status != 404:
                print(f'{url} {status}')
                if status == 200:
                    print(response.headers['content-disposition'])
                    burst_swath = '-'.join(response.headers['content-disposition'].split("filename=")[-1].split('_')[1:3])
                    bursts.append(f"{orbital_path}-{burst_swath}")
                elif status == 202:
                    run = True

In [ ]:
bursts = [b.replace('-', '_') for b in bursts]
print(f"Found {len(bursts)} bursts") 
bursts